# Imports/Installs

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving criminal_lvl2_under200.csv to criminal_lvl2_under200.csv


In [ ]:
pip install simpletransformers

     |████████████████████████████████| 221 kB 8.1 MB/s 
     |████████████████████████████████| 7.9 MB 12.7 MB/s 
     |████████████████████████████████| 1.2 MB 65.1 MB/s 
     |████████████████████████████████| 542 kB 63.1 MB/s 
     |████████████████████████████████| 124 kB 78.8 MB/s 
     |████████████████████████████████| 76 kB 6.2 MB/s 
     |████████████████████████████████| 43 kB 2.5 MB/s 
     |████████████████████████████████| 3.3 MB 58.2 MB/s 
     |████████████████████████████████| 1.8 MB 49.8 MB/s 
     |████████████████████████████████| 2.6 MB 62.1 MB/s 
     |████████████████████████████████| 636 kB 52.3 MB/s 
     |████████████████████████████████| 895 kB 58.1 MB/s 
     |████████████████████████████████| 170 kB 76.0 MB/s 
     |████████████████████████████████| 133 kB 63.4 MB/s 
     |████████████████████████████████| 138 kB 78.0 MB/s 
     |████████████████████████████████| 97 kB 9.3 MB/s 
     |████████████████████████████████| 63 kB 2.2 MB/s 
     |█████████████████

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import torch
import pickle
import sklearn

from simpletransformers.classification import MultiLabelClassificationModel, MultiLabelClassificationArgs
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import classification_report, multilabel_confusion_matrix, precision_recall_fscore_support, hamming_loss, accuracy_score



# Extract data

In [ ]:
data = pd.read_csv('criminal_lvl2_under200.csv', index_col=0)  

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2520 entries, 0 to 2519
Data columns (total 23 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   id                  2520 non-null   int64 
 1   year                2520 non-null   int64 
 2   gr_no_x             2519 non-null   object
 3   text                2520 non-null   object
 4   text_clean          2511 non-null   object
 5   lemma_separated     2520 non-null   object
 6   lvl2_labels_counts  2520 non-null   object
 7   lvl2_labels         2520 non-null   object
 8   chastity            2520 non-null   int64 
 9   civil_status        2520 non-null   int64 
 10  drugs               2520 non-null   int64 
 11  fundamental_laws    2520 non-null   int64 
 12  honor               2520 non-null   int64 
 13  liberty             2520 non-null   int64 
 14  national_security   2520 non-null   int64 
 15  persons             2520 non-null   int64 
 16  property            2520

In [ ]:
data.head(2)

,id,year,gr_no_x,text,text_clean,lemma_separated,lvl2_labels_counts,lvl2_labels,chastity,civil_status,drugs,fundamental_laws,honor,liberty,national_security,persons,property,public_interest,public_morals,public_officers,public_order,quasi_offense,lvl2_max
0,189096,2002,G. R. No. 144340-42,Before this for automatic review[] is the Join...,automatic review joint promulgated h lower cou...,['automatic review joint promulgated h lower c...,"{'national_security': 0, 'fundamental_laws': 0...","['public_interest', 'public_officers', 'person...",1,1,0,0,0,1,0,1,0,1,0,1,0,1,chastity
1,187362,2001,G. R. No. 131203,"SECOND In its decision promulgated on , ,[] th...",second decision promulgated found accused appe...,['second decision promulgated found accused ap...,"{'national_security': 0, 'fundamental_laws': 0...","['public_interest', 'persons', 'liberty', 'cha...",1,1,0,0,0,1,0,1,0,1,0,0,0,1,chastity


# Preprocessing

In [ ]:
lvl2_labels = ['chastity', 
               'civil_status',
               'drugs', 
               'fundamental_laws', 
               'honor', 
               'liberty', 
               'national_security',
               'persons', 
               'property', 
               'public_interest', 
               'public_morals',
               'public_officers', 
               'public_order', 
               'quasi_offense']

In [ ]:
data['labels'] = ''
for x in range(len(data)):
    lbls = []
    for l in lvl2_labels:
        lbls.append(data[l][x])
    data['labels'][x] = lbls

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
data.head(2)

,id,year,gr_no_x,text,text_clean,lemma_separated,lvl2_labels_counts,lvl2_labels,chastity,civil_status,drugs,fundamental_laws,honor,liberty,national_security,persons,property,public_interest,public_morals,public_officers,public_order,quasi_offense,lvl2_max,labels
0,189096,2002,G. R. No. 144340-42,Before this for automatic review[] is the Join...,automatic review joint promulgated h lower cou...,['automatic review joint promulgated h lower c...,"{'national_security': 0, 'fundamental_laws': 0...","['public_interest', 'public_officers', 'person...",1,1,0,0,0,1,0,1,0,1,0,1,0,1,chastity,"[1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1]"
1,187362,2001,G. R. No. 131203,"SECOND In its decision promulgated on , ,[] th...",second decision promulgated found accused appe...,['second decision promulgated found accused ap...,"{'national_security': 0, 'fundamental_laws': 0...","['public_interest', 'persons', 'liberty', 'cha...",1,1,0,0,0,1,0,1,0,1,0,0,0,1,chastity,"[1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1]"


In [ ]:
df = data[['text','labels']]

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2520 entries, 0 to 2519
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    2520 non-null   object
 1   labels  2520 non-null   object
dtypes: object(2)
memory usage: 139.1+ KB


In [ ]:
train_df, eval_df = train_test_split(df, test_size=0.2, random_state=42)

In [ ]:
print(train_df.shape, eval_df.shape)

(2016, 2) (504, 2)


# Model Training (Distilbert-large)

In [ ]:
cuda_available = torch.cuda.is_available()

In [ ]:
model_args = MultiLabelClassificationArgs(learning_rate=4e-5,
                                          adam_epsilon=1e-08,
                                          num_train_epochs=5,
                                          overwrite_output_dir = True)

model = MultiLabelClassificationModel('distilbert', 'distilbert-base-uncased',
                                      use_cuda=cuda_available, 
                                      num_labels=14, 
                                      args=model_args)

Downloading:   0%|          | 0.00/442 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForMultiLabelSequenceClassification: ['vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_projector.weight', 'vocab_projector.bias', 'vocab_transform.weight', 'vocab_layer_norm.bias']
- This IS expected if you are initializing DistilBertForMultiLabelSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForMultiLabelSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForMultiLabelSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
model.train_model(train_df)

  0%|          | 0/2016 [00:00<?, ?it/s]

Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

Running Epoch 0 of 5:   0%|          | 0/252 [00:00<?, ?it/s]

Running Epoch 1 of 5:   0%|          | 0/252 [00:00<?, ?it/s]

Running Epoch 2 of 5:   0%|          | 0/252 [00:00<?, ?it/s]

Running Epoch 3 of 5:   0%|          | 0/252 [00:00<?, ?it/s]

Running Epoch 4 of 5:   0%|          | 0/252 [00:00<?, ?it/s]

(1260, 0.41157119215007815)

In [ ]:
result, model_outputs, wrong_predictions = model.eval_model(eval_df,verbose=True)

  0%|          | 0/504 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/63 [00:00<?, ?it/s]

In [ ]:
result

{'LRAP': 0.8172475210255282, 'eval_loss': 0.40800339976946515}

In [ ]:
model_outputs

array([[0.96337891, 0.84082031, 0.22241211, ..., 0.46972656, 0.01956177,
        0.94042969],
       [0.01589966, 0.14025879, 0.07965088, ..., 0.55322266, 0.00981903,
        0.35302734],
       [0.24707031, 0.77197266, 0.60253906, ..., 0.89501953, 0.05023193,
        0.84472656],
       ...,
       [0.03860474, 0.12792969, 0.08416748, ..., 0.3684082 , 0.02133179,
        0.56103516],
       [0.01024628, 0.11358643, 0.13830566, ..., 0.49902344, 0.01191711,
        0.61230469],
       [0.04995728, 0.52001953, 0.62548828, ..., 0.8515625 , 0.03320312,
        0.91796875]])

#Save model

In [ ]:
import pickle

def save_model(fname, model):
    """
        fname: path/filename.pkl
        model: model

        Saves model to a pickle file
    """
    file = open(fname, 'wb')

    pickle.dump(model, file)

    file.close()

def load_model(fname):
    """
        fname: path/filename.pkl
        Loads a model
    """
    file = open(fname, 'rb')
    data = pickle.load(file)
    file.close()

    return data

In [ ]:
save_model('distilbert.pkl',model)

In [ ]:
#drive.flush_and_unmount()
print('All changes made in this colab session should now be visible in Drive.')

All changes made in this colab session should now be visible in Drive.


In [ ]:
#drive.mount('/content/drive')

Mounted at /content/drive


# Metrics

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
distilbert = load_model('distilbert.pkl')

In [ ]:
result, model_outputs, wrong_predictions = distilbert.eval_model(eval_df,verbose=True)

  0%|          | 0/504 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/63 [00:00<?, ?it/s]

In [ ]:
eval_text = list(eval_df['text'])

In [ ]:
pred, pred_outputs = distilbert.predict(eval_text)

  0%|          | 0/504 [00:00<?, ?it/s]

  0%|          | 0/63 [00:00<?, ?it/s]

In [ ]:
y_pred = list(pred)
y_pred

[[1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
 [0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1],
 [0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1],
 [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1],
 [1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1],
 [0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1],
 [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1],
 [0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1],
 [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0],
 [0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1],
 [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1],
 [0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1],
 [0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1],
 [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0,

In [ ]:
pred_outputs

In [ ]:
y_true = list(eval_df['labels'])
y_true

[[1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1],
 [0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1],
 [0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1],
 [0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1],
 [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1],
 [1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1],
 [0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0],
 [0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0],
 [0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0],
 [0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1],
 [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1],
 [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1],
 [0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1],
 [0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0],
 [0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1],
 [0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0,

In [ ]:
print(precision_recall_fscore_support(y_true, y_pred, average='macro'),  '\n',
      precision_recall_fscore_support(y_true, y_pred, average='micro'), '\n',
      precision_recall_fscore_support(y_true, y_pred, average='weighted'), '\n',
      precision_recall_fscore_support(y_true, y_pred, average='samples', zero_division=1))

(0.7413991938170259, 0.510877941571464, 0.5753778394072157, None) 
 (0.7195053400786959, 0.6006569685593618, 0.6547314578005116, None) 
 (0.7201700841872875, 0.6006569685593618, 0.6370574662739891, None) 
 (0.7630933055040198, 0.6094595329714376, 0.6016030068463541, None)


In [ ]:
accuracy_score(y_true,y_pred)

0.1130952380952381

In [ ]:
top_k = tf.keras.metrics.TopKCategoricalAccuracy(k=2)

In [ ]:
top_k.update_state(y_true,y_pred)
top_k.result().numpy()

0.7361111

In [ ]:
top_k = tf.keras.metrics.TopKCategoricalAccuracy(k=3)
top_k.update_state(y_true,y_pred)
top_k.result().numpy()

0.8035714

In [ ]:
top_k = tf.keras.metrics.TopKCategoricalAccuracy(k=4)
top_k.update_state(y_true,y_pred)
top_k.result().numpy()

0.859127

In [ ]:
top_k = tf.keras.metrics.TopKCategoricalAccuracy(k=1)
top_k.update_state(y_true,y_pred)
top_k.result().numpy()

0.6626984

# Sample Prediction

In [ ]:
sample_text = ['Avelino Ordono was charged in the municipal court of San Gabriel, La Union with having raped his daughter, Leonora, on October 11, 1970. The verified complaint dated November 7, 1973 was signed by the twenty-four year old victim (Criminal Case No. 104).In support of that complaint, Catalina Balanon Ordono, the mother of Leonora, executed a sworn statement wherein she disclosed that on that same date, October 11th, Leonora had apprised her of the outrage but no denunciation was filed because Avelino Ordono threatened to kill Leonora and Catalina (his daughter and wife, respectively) if they reported the crime to the police.Catalina Ordono in her sworn statement further revealed that her husband had also raped their other daughter, Rosa, on March 25 and April 7, 1973. He was charged in court with that offense.Catalina Ordono said that the rape committed by Avelino Ordono against Leonora was mentioned during the investigation and trial of Avelino Ordono for the rape committed against Rosa Ordono. Catalina\'s statement on this point is as follows:  "Q - Why did you not file the complaint against your husband concerning the incident involving Leonora Ordono?"A - We also narrated the incident during the investigation in the Fiscal\'s Office and also when I testified in court in the case of my daughter Rosa Ordono but then my daughter Leonora Ordono was still in Manila, sir."During the preliminary investigation of the rape committed against Leonora, Catalina manifested that she was no longer afraid to denounce Avelino Ordono because he was already in jail for having raped Rosa Ordono.The case against Avelino Ordono, where Leonora Ordono was the complainant, was elevated to the Court of First Instance of La Union, San Fernando Branch (Criminal Case No. 356). On May 29, 1974 the Fiscal presented Catalina Ordono as the second prosecution witness. After she had stated her personal circumstances, the defense counsel objected to her competency. He invoked the marital disqualification rule found in Rule 130 of the Rules of Court which provides:"SEC. 20.  Disqualification by reason of interest or relationship. — The following persons cannot testify as to matters in which they are interested, directly or indirectly, as herein enumerated:x x x x x x x x x(b) A husband cannot be examined for or against his wife without her consent; nor a wife for or against her husband without his consent, except in a civil case by one against the other, or in a criminal case for a crime committed by one against the other;x x x x x x x x xCounsel claimed that Avelino Ordono had not consented expressly or impliedly to his wife\'s testifying against him.The trial court overruled the objection. After the denial of Avelino Ordono\'s motion for the reconsideration of the adverse ruling, he filed the instant action for certiorari and prohibition. He was allowed to sue in forma pauperis.The issue is whether the rape committed by the husband against his daughter is a crime committed by him against his wife within the meaning of the exception found in the marital disqualification rule.Should the phrase "in a criminal case for a crime committed by one against the other" be restricted to crimes committed by one spouse against the other, such as physical injuries, bigamy, adultery or concubinage, or should it be given a latitudinarian interpretation as referring to any offense causing marital discord?There is a dictum that "where the marital and domestic relations are so strained that there is no more harmony to be preserved nor peace and tranquility which may be disturbed, the reason based upon such harmony and tranquility fails. In such a case identity of interests disappears and the consequent danger of perjury based on that identity is nonexistent. Likewise, in such a situation, the security and confidences of private life which the law aims at protecting will be nothing but ideals which, through their absence, merely leave a void in the unhappy home" (People vs. Francisco, 78 Phil. 694, 704).In the Francisco case, the wife, as a rebuttal witness, was allowed to testify against the husband who was charged with having killed his son and who testified that it was the wife who killed their son.We think that the correct rule, which may be adopted in this jurisdiction, is that laid down in Cargill vs. State, 35 ALR 133, 220 Pac. 64 25 Okl. 314, wherein the court said:"The rule that the injury must amount to a physical wrong upon the person is too narrow; and the rule that any offense remotely or indirectly affecting domestic harmony comes within the exception is too broad. The better rule is that, when an offense directly attacks, or directly and vitally impairs, the conjugal relation, it comes within the exception to the statute that one shall not be a witness against the other except in a criminal prosecution for a crime committed (by) one against the other".Using the criterion thus judiciously enunciated in the Cargill case, it can be conclude that in the law of evidence the rape perpetrated by the father against his daughter is a crime committed by him against his wife (the victim\'s mother).[*]That conclusion is in harmony with the practices and traditions of the Filipino family where, normally, the daughter is close to the mother who, having breast-fed and reared her offspring, is always ready to render her counsel and assistance in time of need. Indeed, when the daughter is in distress or suffers moral or physical pain, she usually utters the word Inay (Mother) before she invokes the name of the Lord. (Madre mia, Dios mio)Thus, in this case, when Avelino Ordono, after having raped his daughter Leonora in the early morning of October 11, 1970, tried to repeat the beastly act in the evening of that date, Leonora shouted "Mother" and, on hearing that word, Avelino desisted.That the rape of the daughter by the father, an undeniably abominable and revolting crime with incestuous implications, positively undermines the connubial relationship, is a proposition too obvious to require much elucidation.In Wilkinson vs. People, 282 Pac. 257, it was held that the wife was a competent witness against the husband in a prosecution for rape committed by the husband against his stepdaughter, who is the wife\'s natural daughter because the crime was "an outrage upon nature in its dearest and tenderest relations as well as a crime against humanity itself". The court adopted the interpretation that "a criminal action or proceeding for a crime committed by one against the other" may refer to a crime where the wife is the individual particularly and directly injured or affected by the crime for which the husband is being prosecuted (See Dill vs. People, 19 Colo. 469, 475, 36 Pac. 229, 232).In State vs. Chambers, 87 Iowa 1, 53 N.W. 1090, it was held under the statutory provision that husband or wife shall in no case be a witness for or against the other, except in a criminal proceeding for a crime committed by one against the other, that the wife was competent to testify against the husband in a case where he was prosecuted for incest committed against his stepdaughter.In State vs. Shultz, 177 Iowa 321, 158 N.W. 539, it was held that the wife may testify against the husband in a case where he was prosecuted for incest committed against their eleven-year old daughter because incest is a "crime committed against the wife". (See Owens vs. State, 32 Neb. 167, 49 N.W. 226; Lord vs. State, 23 N.W. 507, 17 Neb. 526; People vs. Segura, 60 Phil. 933).The trial court did not err in holding that Catalina Ordono could testify against her husband, Avelino Ordono, in the case where he is being tried for having raped their daughter, Leonora.WHEREFORE, the petition for certiorari and prohibition is dismissed. No costs.SO ORDERED.Makalintal, C.J., Castro, Fernando, Teehankee, Barredo, Makasiar, Esguerra, Fernandez, and Muñoz Palma, JJ., concur.Antonio, J., is on sick leave.[*] As noted by Wigmore, the marital disqualification rule consists of two parts: (1) the incapacity of one spouse to testify for the other, a disqualification designed to obviate perjury, and (2) the privilege of one spouse not to testify against the other, a right designed to present domestic disunion and unhappiness (U.S. vs. Concepcion, 31 Phil. 182, 187; 2 Wigmore on Evidence 731).Wigmore notes that the privilege has no longer any good reason for retention. "In an age which has so far rationalized, depolarized, and depolarized the marital relation and the spirit of Femininity as to be willing to enact complete legal and political equality and independence of man and woman, this marital privilege is the merest anachronism, in legal theory, and an indefensible obstruction to truth, in practice". After noting that some States had abolished the privilege, the American Bar Association\'s Committee on the Improvement of the Law of Evidence in 1937-38 recommended its abolition (8 Wigmore on Evidence 232).']
print(sample_text[0])

Avelino Ordono was charged in the municipal court of San Gabriel, La Union with having raped his daughter, Leonora, on October 11, 1970. The verified complaint dated November 7, 1973 was signed by the twenty-four year old victim (Criminal Case No. 104).In support of that complaint, Catalina Balanon Ordono, the mother of Leonora, executed a sworn statement wherein she disclosed that on that same date, October 11th, Leonora had apprised her of the outrage but no denunciation was filed because Avelino Ordono threatened to kill Leonora and Catalina (his daughter and wife, respectively) if they reported the crime to the police.Catalina Ordono in her sworn statement further revealed that her husband had also raped their other daughter, Rosa, on March 25 and April 7, 1973. He was charged in court with that offense.Catalina Ordono said that the rape committed by Avelino Ordono against Leonora was mentioned during the investigation and trial of Avelino Ordono for the rape committed against Rosa

In [ ]:
prediction, prediction_outputs = distilbert.predict(sample_text)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
prediction[0]

[1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1]

In [ ]:
prediction_outputs[0]

array([0.91748047, 0.80175781, 0.12780762, 0.19995117, 0.50292969,
       0.73535156, 0.00945282, 0.78857422, 0.07904053, 0.25      ,
       0.03671265, 0.40771484, 0.01111603, 0.88769531])

In [ ]:
def label_ranked(probs):
    labels = list(zip(lvl2_labels,probs))
            
    return sorted(labels, key = lambda i: i[1],reverse=True)

In [ ]:
prediction_labels_ranked = label_ranked(prediction_outputs[0])

In [ ]:
prediction_labels_ranked

[('chastity', 0.91748046875),
 ('quasi_offense', 0.8876953125),
 ('civil_status', 0.8017578125),
 ('persons', 0.78857421875),
 ('liberty', 0.7353515625),
 ('honor', 0.5029296875),
 ('public_officers', 0.40771484375),
 ('public_interest', 0.25),
 ('fundamental_laws', 0.199951171875),
 ('drugs', 0.1278076171875),
 ('property', 0.07904052734375),
 ('public_morals', 0.036712646484375),
 ('public_order', 0.01111602783203125),
 ('national_security', 0.00945281982421875)]

In [ ]:
import textwrap
  
value = """This function wraps the input paragraph such that each line
in the paragraph is at most width characters long. The wrap method
returns a list of output lines. The returned list
is empty if the wrapped
output has no content."""
  
# Wrap this text.
wrapper = textwrap.TextWrapper(width=50)
  
word_list = wrapper.wrap(text=value)
  
# Print each line.
for element in word_list:
    print(element)

This function wraps the input paragraph such that
each line in the paragraph is at most width
characters long. The wrap method returns a list of
output lines. The returned list is empty if the
wrapped output has no content.


# Eval set predictions

In [ ]:
df2 = data[['gr_no_x', 'text', 'labels', 'lvl2_max']]

In [ ]:
df2.head()

,gr_no_x,text,labels,lvl2_max
0,G. R. No. 144340-42,Before this for automatic review[] is the Join...,"[1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1]",chastity
1,G. R. No. 131203,"SECOND In its decision promulgated on , ,[] th...","[1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1]",chastity
2,G.R. Nos.102409-10,"de appeals from the dated , in and of the , of...","[1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1]",chastity
3,G.R. No. 133926,"For automatic review is the dated , , of the ,...","[1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1]",chastity
4,G. R. No. 139231,"On appeal is the decision[] of the , in ,-, co...","[1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1]",chastity


In [ ]:
train_df2, eval_df2 = train_test_split(df2, test_size=0.2, random_state=42)

In [ ]:
pred_text = list(eval_df2['text'])
#print(sample_text[0])

In [ ]:
prediction, prediction_outputs = distilbert.predict(pred_text)

  0%|          | 0/504 [00:00<?, ?it/s]

  0%|          | 0/63 [00:00<?, ?it/s]

In [ ]:
prediction_outputs

array([[0.96337891, 0.84082031, 0.22241211, ..., 0.46972656, 0.01956177,
        0.94042969],
       [0.01589966, 0.14025879, 0.07965088, ..., 0.55322266, 0.00981903,
        0.35302734],
       [0.24707031, 0.77197266, 0.60253906, ..., 0.89501953, 0.05023193,
        0.84472656],
       ...,
       [0.03860474, 0.12792969, 0.08416748, ..., 0.3684082 , 0.02133179,
        0.56103516],
       [0.01024628, 0.11358643, 0.13830566, ..., 0.49902344, 0.01191711,
        0.61230469],
       [0.04995728, 0.52001953, 0.62548828, ..., 0.8515625 , 0.03320312,
        0.91796875]])

In [ ]:
prediction_labels_ranked = [label_ranked(i) for i in prediction_outputs]

In [ ]:
prediction_labels_ranked

[[('chastity', 0.96337890625),
  ('quasi_offense', 0.9404296875),
  ('liberty', 0.8642578125),
  ('civil_status', 0.8408203125),
  ('persons', 0.8017578125),
  ('honor', 0.62158203125),
  ('public_officers', 0.4697265625),
  ('public_interest', 0.398681640625),
  ('drugs', 0.222412109375),
  ('fundamental_laws', 0.2139892578125),
  ('property', 0.1405029296875),
  ('public_morals', 0.06243896484375),
  ('public_order', 0.019561767578125),
  ('national_security', 0.0169830322265625)],
 [('public_officers', 0.55322265625),
  ('honor', 0.446044921875),
  ('quasi_offense', 0.35302734375),
  ('fundamental_laws', 0.1951904296875),
  ('public_interest', 0.1890869140625),
  ('civil_status', 0.1402587890625),
  ('property', 0.11676025390625),
  ('liberty', 0.115966796875),
  ('drugs', 0.07965087890625),
  ('persons', 0.034820556640625),
  ('public_morals', 0.024749755859375),
  ('chastity', 0.015899658203125),
  ('national_security', 0.0125732421875),
  ('public_order', 0.00981903076171875)],
 

In [ ]:
#eval_df2['lvl2_max_roberta'] = [x[0][0] for x in prediction_labels_ranked]
eval_df2['max_distilbert'] = [x[0][0] for x in prediction_labels_ranked]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
#del df['column name']
del eval_df2['max_roberta']

KeyError: ignored

In [ ]:
eval_df2.head()

,gr_no_x,text,labels,lvl2_max,max_distilbert
196,G.R. Nos. 146352-56,Not only did the accused rape his daughter-in-...,"[1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1]",chastity,chastity
986,A.M. No. MTJ-90-400,", I S I , filed a sworn complaint with the aga...","[0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1]",honor,public_officers
1090,G.R. Nos. 154150-51,For our resolution is a petition for certiorar...,"[0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1]",liberty,public_officers
387,G.R. No. 89278,THIRD This is an appeal from the decision of t...,"[0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1]",civil_status,persons
2221,A.C. No. 1753,"SECOND No. , , A. OF I S I was charged with sl...","[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1]",public_officers,property


In [ ]:
sample_df = eval_df2.drop(columns=['text','labels'])
sample_df.head()

,gr_no_x,lvl2_max,max_distilbert
196,G.R. Nos. 146352-56,chastity,chastity
986,A.M. No. MTJ-90-400,honor,public_officers
1090,G.R. Nos. 154150-51,liberty,public_officers
387,G.R. No. 89278,civil_status,persons
2221,A.C. No. 1753,public_officers,property


In [ ]:
y_pred_outputs = list(pred_outputs)
#y_pred_outputs
y_true = list(eval_df['labels'])
#y_true
print(coverage_error(y_true, y_pred_outputs),  '\n',
      label_ranking_average_precision_score(y_true, y_pred_outputs), '\n',
      label_ranking_loss(y_true, y_pred_outputs))

NameError: ignored

In [ ]:
sample_df.to_csv("DBertFINAL.csv")

In [ ]:
from google.colab import files
files.download("DBertFINAL.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>